<a href="https://colab.research.google.com/github/sangwonkook/kook/blob/main/%EB%A6%AC%EB%B7%B0_%EC%9A%94%EC%95%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# 파일 경로 설정
file_path = '/content/drive/MyDrive/수업진행프로젝트/foreign_worker_reviews_dummy_fixed.csv'

In [1]:
# 1. 라이브러리 설치
!pip install transformers sentencepiece

import pandas as pd
from transformers import pipeline

# 2. 데이터 불러오기
df = pd.read_csv("/content/drive/MyDrive/수업진행프로젝트/foreign_worker_reviews_dummy_fixed.csv")

# 3. 기업별 설문(수치형) 요약: 평균 값 집계
survey_summary = df.groupby("기업명")[["평점", "복지", "임금", "근로환경", "안전", "소통"]].mean().round(2)
print("📊 기업별 평균 점수 요약")
print(survey_summary)

# 4. 리뷰 요약 준비
# 리뷰를 기업별로 합쳐서 요약 진행
company_reviews = df.groupby("기업명")["리뷰내용"].apply(lambda x: " ".join(x)).reset_index()

# 5. 다국어 요약 모델 로드
summarizer = pipeline("summarization", model="facebook/mbart-large-cc25")

# 6. 기업별 리뷰 요약
summaries = []
for _, row in company_reviews.iterrows():
    company = row["기업명"]
    text = row["리뷰내용"]

    # 모델 입력 제한(길이 초과 방지)
    text = text[:1000]

    summary = summarizer(text, max_length=60, min_length=10, do_sample=False)
    summaries.append({"기업명": company, "리뷰요약": summary[0]['summary_text']})

summary_df = pd.DataFrame(summaries)

print("\n📝 리뷰 요약 결과")
print(summary_df)


📊 기업별 평균 점수 요약
           평점    복지    임금  근로환경    안전    소통
기업명                                        
광주물류     3.53  3.35  2.71  2.94  3.00  3.59
광주섬유     3.26  3.22  2.52  3.43  3.26  3.09
광주식품     3.00  3.25  3.44  3.00  3.25  2.88
광주자동차부품  3.21  3.00  3.26  3.05  3.53  3.00
광주전자     2.72  2.64  3.16  3.12  3.16  2.92


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu



📝 리뷰 요약 결과
       기업명                                               리뷰요약
0     광주물류  salary is good but communication is good but c...
1     광주섬유  is not well managed. Thời gian làm việc làm vi...
2     광주식품  gian làm việc làm việc làm việc làm việc làm v...
3  광주자동차부품  andandantsantsantsantsants,但是环境还可以。工资不高,安全,有有有...
4     광주전자  salary is good but communication is good but c...
